In [177]:
import math
import copy
import itertools
import random
import numpy as np
import pandas as pd
import networkx as nx
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl
import dgl.function as fn
from rdkit import Chem
from rdkit.Chem.Crippen import MolLogP, MolMR
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, KFold

# =============================================================================
# Helper Functions
# =============================================================================

def get_hard_soft_value(smile, dataframe):
    """
    Get the 'Hard/Soft' value from the dataframe based on the smile string.
    """
    filtered_df = dataframe[dataframe['Big_Smile'] == smile]
    return filtered_df.iloc[0]['Hard/Soft'] if not filtered_df.empty else None

def graph_features(mol, atom_labels, max_length=None):
    """
    Compute one‐hot features for the atoms in a molecule.
    Pads the feature matrix up to max_length.
    """
    max_length = max_length or mol.GetNumAtoms()
    features = np.array([[a.GetAtomicNum() == i for i in atom_labels] for a in mol.GetAtoms()], dtype=np.int32)
    padding = np.zeros((max_length - features.shape[0], features.shape[1]))
    return np.vstack((features, padding))

def feature_size(mol, atom_labels, max_atoms=60):
    """
    Create a tensor of size (max_atoms x len(atom_labels)) with one-hot encoding for each atom.
    """
    features = torch.zeros((max_atoms, len(atom_labels)))
    for i, atom in enumerate(mol.GetAtoms()):
        if i >= max_atoms:
            break
        try:
            idx = atom_labels.index(atom.GetAtomicNum())
        except ValueError:
            continue  # In case the atomic number is not in atom_labels.
        features[i, idx] = 1
    return features

def graph_adjacency(mol, max_atoms=60, bond_encoder=None):
    """
    Create an adjacency matrix (as a torch tensor) for the molecule.
    Each edge is weighted by the encoded bond type.
    """
    bond_encoder = bond_encoder or {}
    adjacency = torch.zeros((max_atoms, max_atoms))
    for bond in mol.GetBonds():
        start, end = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
        if start < max_atoms and end < max_atoms:
            weight = bond_encoder.get(bond.GetBondType(), 0)
            adjacency[start, end] = weight
            adjacency[end, start] = weight
    return adjacency

def find_paths(G, u, n):
    """
    Recursively find all simple paths of length n from node u in graph G.
    """
    if n == 0:
        return [[u]]
    return [
        [u] + path
        for neighbor in G.neighbors(u)
        for path in find_paths(G, neighbor, n - 1)
        if u not in path
    ]

def find_minimum_indices(arr):
    """
    Return the indices where the minimum value occurs in the list.
    """
    min_val = min(arr)
    return [index for index, value in enumerate(arr) if value == min_val]

def remove_unique_classes(class_matrix, feature_matrix):
    """
    Remove rows from class_matrix and feature_matrix where the class is unique.
    """
    unique_classes, class_counts = np.unique(class_matrix, axis=0, return_counts=True)
    unique_indices = np.where(class_counts == 1)[0]
    return np.delete(class_matrix, unique_indices, axis=0), np.delete(feature_matrix, unique_indices, axis=0)

def map_all_edges_g_to_G(g, G):
    """
    Map DGL graph edges to the original NetworkX graph node names.
    """
    edge_to_str_name_mapping = {}
    for edge_id in range(g.number_of_edges()):
        src_id, dst_id = g.find_edges(edge_id)
        src_name = g.ndata['name'][src_id].item()
        dst_name = g.ndata['name'][dst_id].item()
        src_in_G, dst_in_G = None, None
        for node, data in G.nodes(data=True):
            if data.get('name') == src_name:
                src_in_G = node
            if data.get('name') == dst_name:
                dst_in_G = node
            if src_in_G is not None and dst_in_G is not None:
                break
        if src_in_G is not None and dst_in_G is not None:
            edge_to_str_name_mapping[edge_id] = (src_in_G, dst_in_G)
    return edge_to_str_name_mapping

# =============================================================================
# Data Extraction & Graph Construction Function
# =============================================================================

def DataExtN():
    df = pd.read_excel('database.xlsx', 'BCPs')
    dfH = pd.read_excel('database.xlsx', 'Homopolymers')
    print('- Total BCP data:', df.shape[0])
    print('- Total Homopolymer data:', dfH.shape[0])

    imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
    # XE = [df['σbreak (MPa)']]
    # XE1 = [df['εbreak (%)']]
    XE = [df['εbreak (%)']]
    XE1 = [df['εbreak (%)']]

    smiles = df['Big_Smile'].tolist()
    counts = df['Block DP']
    features, features1, features2 = [], [], []
    G = nx.DiGraph()

    # Extract features for each SMILES entry
    for i in range(len(smiles)):
        features.append([XE[0][i]])
        features1.append([XE[0][i]])
        features2.append([XE1[0][i]])
    features = imputer.fit_transform(features)

    Count = 0
    for i in range(len(smiles)):
        Sp = smiles[i][1:-1].split('}{')
        Cn = str(counts[i]).split(':')
        if len(Cn[0].split('/')) == 1:
            Cn[0] = str(2 * float(Cn[0]))
        else:
            AC = Cn[0].split('/')
            Cn[0] = str(2 * float(AC[0])) + '/' + str(2 * float(AC[1]))

        # Expecting two segments
        Sp = [Sp[0], Sp[1]]
        Spp = []
        for l in range(2):
            if len(Sp[l].split(',')) == 1:
                Spp.append(Sp[l])

        data = [Chem.MolFromSmiles(line) for line in Spp if Chem.MolFromSmiles(line) is not None]
        if not data:
            continue  # Skip if no valid molecule is generated
        atom_labels = sorted(set([atom.GetAtomicNum() for mol in data for atom in mol.GetAtoms()] + [0]))
        bond_labels = [Chem.rdchem.BondType.ZERO] + list(sorted(set(bond.GetBondType() for mol in data for bond in mol.GetBonds())))
        bond_encoder_m = {l: ii for ii, l in enumerate(bond_labels)}

        Feat = []
        for idx, mol in enumerate(data):
            combined_feature = torch.cat([feature_size(mol, atom_labels),
                                          graph_adjacency(mol, bond_encoder=bond_encoder_m)], dim=1)
            DP_values = Cn[idx].split('/')
            dp_weight = torch.sqrt(torch.tensor(float(DP_values[0]), dtype=torch.float))
            weighted_features = (combined_feature + 1) * dp_weight
            Feat.append(weighted_features.float())

        Feat = torch.stack(Feat)
        Feat = F.pad(Feat, (0, 65 - Feat.size(-1)), "constant", 0)

        # Process sub-features if any segment has a comma
        if len(Sp[0].split(',')) > 1 or len(Sp[1].split(',')) > 1:
            if len(Sp[0].split(',')) > 1:
                ss = Sp[0].split(',')
                jj = Cn[0].split('/')
            if len(Sp[1].split(',')) > 1:
                ss = Sp[1].split(',')
                jj = Cn[1].split('/')
            data = [Chem.MolFromSmiles(ss[0])]
            atom_labels = sorted(set([atom.GetAtomicNum() for mol in data for atom in mol.GetAtoms()] + [0]))
            bond_labels = [Chem.rdchem.BondType.ZERO] + list(sorted(set(bond.GetBondType() for mol in data for bond in mol.GetBonds())))
            bond_encoder_m = {l: ii for ii, l in enumerate(bond_labels)}
            Feat1 = []
            for idx, mol in enumerate(data):
                combined_feature = torch.cat([feature_size(mol, atom_labels),
                                              graph_adjacency(mol, bond_encoder=bond_encoder_m)], dim=1)
                dp_weight = torch.sqrt(torch.tensor(float(jj[0]), dtype=torch.float))
                weighted_features = (combined_feature + 1) * dp_weight
                Feat1.append(weighted_features.float())
            Feat1 = torch.stack(Feat1)
            Feat1 = F.pad(Feat1, (0, 65 - Feat1.size(-1)), "constant", 0)

            data = [Chem.MolFromSmiles(ss[1])]
            atom_labels = sorted(set([atom.GetAtomicNum() for mol in data for atom in mol.GetAtoms()] + [0]))
            bond_labels = [Chem.rdchem.BondType.ZERO] + list(sorted(set(bond.GetBondType() for mol in data for bond in mol.GetBonds())))
            bond_encoder_m = {l: ii for ii, l in enumerate(bond_labels)}
            Feat2 = []
            for idx, mol in enumerate(data):
                combined_feature = torch.cat([feature_size(mol, atom_labels),
                                              graph_adjacency(mol, bond_encoder=bond_encoder_m)], dim=1)
                dp_weight = torch.sqrt(torch.tensor(float(jj[1]), dtype=torch.float))
                weighted_features = (combined_feature + 1) * dp_weight
                Feat2.append(weighted_features.float())
            Feat2 = torch.stack(Feat2)
            Feat2 = F.pad(Feat2, (0, 65 - Feat2.size(-1)), "constant", 0)
            Feat3 = Feat1 + Feat2
            Feat = torch.cat((Feat, Feat3), dim=0)

        # Create padded features for the node (flatten and pad to fixed length 4000)
        padded_features = []
        for feat in Feat:
            feat_flat = feat.reshape(-1)
            pad_len = 4000 - feat_flat.shape[0]
            feat_flat = torch.cat([feat_flat, torch.zeros(pad_len)])
            padded_features.append(feat_flat)
        padded_features = torch.stack(padded_features)
        # --- FIX: Ensure we have at least two feature vectors ---
        if padded_features.shape[0] < 2:
            padded_features = torch.cat([padded_features, padded_features], dim=0)

        Feat_T = torch.tensor(features[i])
        Feat_TE = torch.tensor(features1[i])
        Feat_TE1 = torch.tensor(features2[i])

        value0 = get_hard_soft_value(Sp[0], dfH)
        value1 = get_hard_soft_value(Sp[1], dfH)

        node_name0 = Sp[0] + ';' + Cn[0]
        node_name1 = Sp[1] + ';' + (Cn[1] if len(Cn) > 1 else Cn[0])

        if value0 is not None and value1 is None:
            if not G.has_node(node_name0):
                Count += 1
                G.add_node(node_name0, Feature=padded_features[0],
                           weight=Feat_T, bipartite=value0,
                           weightBreak=Feat_TE1, name=Count)
            if not G.has_node(node_name1):
                Count += 1
                G.add_node(node_name1, Feature=padded_features[1],
                           weight=Feat_T, bipartite=1 - value0,
                           weightBreak=Feat_TE1, name=Count)
        elif value1 is not None and value0 is None:
            if not G.has_node(node_name0):
                Count += 1
                G.add_node(node_name0, Feature=padded_features[0],
                           weight=Feat_T, bipartite=1 - value1,
                           weightBreak=Feat_TE1, name=Count)
            if not G.has_node(node_name1):
                Count += 1
                G.add_node(node_name1, Feature=padded_features[1],
                           weight=Feat_T, bipartite=value1,
                           weightBreak=Feat_TE1, name=Count)
        elif value0 is not None and value1 is not None:
            if not G.has_node(node_name0):
                Count += 1
                G.add_node(node_name0, Feature=padded_features[0],
                           weight=Feat_T, bipartite=value0,
                           weightBreak=Feat_TE1, name=Count)
            if not G.has_node(node_name1):
                Count += 1
                G.add_node(node_name1, Feature=padded_features[1],
                           weight=Feat_T, bipartite=value1,
                           weightBreak=Feat_TE1, name=Count)
        else:
            if not G.has_node(node_name0):
                Count += 1
                G.add_node(node_name0, Feature=padded_features[0],
                           weight=Feat_T, bipartite=0,
                           weightBreak=Feat_TE1, name=Count)
            if not G.has_node(node_name1):
                Count += 1
                G.add_node(node_name1, Feature=padded_features[1],
                           weight=Feat_T, bipartite=1,
                           weightBreak=Feat_TE1, name=Count)

        G.add_edge(node_name0, node_name1, Feature=padded_features[0],
                   weight=Feat_TE, weightBreak=Feat_TE1)

    mapping = {node: i for i, node in enumerate(G.nodes())}
    reverse_mapping = {i: node for node, i in mapping.items()}
    dgl_g = dgl.from_networkx(G, node_attrs=['Feature','weight','weightBreak','bipartite','name'],
                              edge_attrs=['Feature','weight','weightBreak'])
    return dgl_g, G, reverse_mapping


# =============================================================================
# Graph Neural Network Model Definitions
# =============================================================================

class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GraphSAGE, self).__init__()
        self.conv1 = dgl.nn.SAGEConv(in_feats, h_feats, 'mean')
        self.dropout = nn.Dropout(0.5)  # Dropout rate 0.5
        self.conv2 = dgl.nn.SAGEConv(h_feats, h_feats, 'mean')
        self.conv3 = dgl.nn.SAGEConv(h_feats, h_feats, 'mean')
        self.conv4 = dgl.nn.SAGEConv(h_feats, h_feats, 'mean')

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv2(g, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv3(g, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv4(g, h)
        return h

class DotPredictor(nn.Module):
    def forward(self, g, h):
        g.ndata['h'] = h
        g.apply_edges(fn.u_dot_v('h', 'h', 'score'))
        # Return scores as a 1D tensor along with the updated graph
        return g.edata['score'][:, 0], g

def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0.01)
    elif hasattr(m, 'fc_self'):
        nn.init.xavier_uniform_(m.fc_self.weight)
        if m.fc_self.bias is not None:
            m.fc_self.bias.data.fill_(0.01)
    elif hasattr(m, 'fc_neigh'):
        nn.init.xavier_uniform_(m.fc_neigh.weight)
        if m.fc_neigh.bias is not None:
            m.fc_neigh.bias.data.fill_(0.01)

def compute_loss(pos_score, neg_score, weight_pos=1.0, weight_neg=1.0):
    pos_loss = F.binary_cross_entropy_with_logits(
        pos_score, torch.ones_like(pos_score),
        weight=torch.full_like(pos_score, weight_pos)
    )
    neg_loss = F.binary_cross_entropy_with_logits(
        neg_score, torch.zeros_like(neg_score),
        weight=torch.full_like(neg_score, weight_neg)
    )
    return pos_loss + neg_loss

def compute_auc(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score]).detach().cpu().numpy()
    labels = torch.cat([torch.ones(pos_score.shape[0]),
                        torch.zeros(neg_score.shape[0])]).detach().cpu().numpy()
    return roc_auc_score(labels, scores)

def calculate_rmse(pred_scores, true_weights):
    mse = np.mean((pred_scores.numpy() - true_weights.numpy()) ** 2)
    return math.sqrt(mse)

# =============================================================================
# Main Training and Evaluation Routine
# =============================================================================

def main():
    seed = 2
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # Load the data and construct graphs
    g, G, reverse_mapping = DataExtN()
    u, v = g.edges()
    eids = np.arange(g.number_of_edges())

    # Generate negative edges using bipartite node subset
    bipartite_nodes = np.where(g.ndata["bipartite"] == 1)[0]
    current_edges = set(zip(u.numpy(), v.numpy()))
    possible_neg_edges = [(x, y) for x in bipartite_nodes for y in bipartite_nodes if x != y]
    neg_edges = [edge for edge in possible_neg_edges if edge not in current_edges]
    if not neg_edges:
        raise ValueError("No negative edges found. Check your bipartite node definitions.")
    neg_u, neg_v = zip(*neg_edges)

    # Set up 2-fold cross-validation
    kf = KFold(n_splits=2, shuffle=True, random_state=seed)

    for fold, (train_val_index, test_index) in enumerate(kf.split(eids)):
        train_index, val_index = train_test_split(train_val_index, test_size=0.5, random_state=seed)
        train_size = len(train_index)
        val_size = len(val_index)
        test_size = len(test_index)

        # Select positive edges for train/validation/test
        train_pos_u, train_pos_v = u[train_index], v[train_index]
        val_pos_u, val_pos_v = u[val_index], v[val_index]
        test_pos_u, test_pos_v = u[test_index], v[test_index]

        # Sample negative edges to match the number of positive edges
        neg_indices = np.random.permutation(len(neg_u))
        train_neg_u = np.array(neg_u)[neg_indices[:train_size]]
        train_neg_v = np.array(neg_v)[neg_indices[:train_size]]
        val_neg_u = np.array(neg_u)[neg_indices[train_size:train_size + val_size]]
        val_neg_v = np.array(neg_v)[neg_indices[train_size:train_size + val_size]]
        test_neg_u = np.array(neg_u)[neg_indices[train_size + val_size:train_size + val_size + test_size]]
        test_neg_v = np.array(neg_v)[neg_indices[train_size + val_size:train_size + val_size + test_size]]

        # Create training graph by removing a subset of edges from the full graph
        train_g = dgl.remove_edges(g, eids[:train_size + val_size])

        # Build subgraphs for positive edges
        def build_subgraph(pos_u, pos_v):
            sub_g = dgl.graph((pos_u, pos_v), num_nodes=g.number_of_nodes())
            sub_g.ndata.update(g.ndata)
            return sub_g

        train_pos_g = build_subgraph(train_pos_u, train_pos_v)
        val_pos_g = build_subgraph(val_pos_u, val_pos_v)
        test_pos_g = build_subgraph(test_pos_u, test_pos_v)

        # Build subgraphs for negative edges
        train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.number_of_nodes())
        train_neg_g.ndata.update(g.ndata)
        val_neg_g = dgl.graph((val_neg_u, val_neg_v), num_nodes=g.number_of_nodes())
        val_neg_g.ndata.update(g.ndata)
        test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.number_of_nodes())
        test_neg_g.ndata.update(g.ndata)

        # Initialize model and predictor
        in_feats = g.ndata['Feature'].shape[1]
        model = GraphSAGE(in_feats, 32)
        model.apply(init_weights)
        pred = DotPredictor()

        optimizer = torch.optim.Adam(
            itertools.chain(model.parameters(), pred.parameters()),
            lr=0.001, weight_decay=1e-4
        )
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.1, patience=10)

        best_val_loss = float('inf')
        patience = 1000
        no_improvement_count = 0

        # Training loop
        for e in range(500):
            model.train()
            h = model(train_g, train_g.ndata['Feature'].float())
            pos_score, _ = pred(train_pos_g, h)
            neg_score, _ = pred(train_neg_g, h)
            loss = compute_loss(pos_score, neg_score)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            model.eval()
            with torch.no_grad():
                val_h = model(train_g, train_g.ndata['Feature'].float())
                val_pos_score, _ = pred(val_pos_g, val_h)
                val_neg_score, _ = pred(val_neg_g, val_h)
                val_loss = compute_loss(val_pos_score, val_neg_score)

            if val_loss < best_val_loss:
                best_val_loss = val_loss
                torch.save(model.state_dict(), "best_model.pth")
                no_improvement_count = 0
            else:
                no_improvement_count += 1

            if e % 20 == 0:
                print(f'Epoch {e}, Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')

            if no_improvement_count >= patience:
                print("Early stopping due to no improvement")
                break

        # Evaluation on test set
        model.eval()
        with torch.no_grad():
            test_h = model(test_pos_g, test_pos_g.ndata['Feature'].float())
            pos_score, _ = pred(test_pos_g, test_h)
            neg_score, _ = pred(test_neg_g, test_h)
            auc = compute_auc(pos_score, neg_score)
            print(f'Fold {fold + 1}, AUC: {auc:.4f}')

            # Predict on the full graph and map edges back to original names
            full_score, gF = pred(g, test_h)
            edge_to_str_name_mapping = map_all_edges_g_to_G(gF, G)

            predicted_scores = []
            true_weights = []
            with open('predicted_edges.txt', 'w') as f:
                f.write("Source,Target,Score,Weight\n")
                for edge_id, (src_name, dst_name) in edge_to_str_name_mapping.items():
                    predicted_score = gF.edges[edge_id].data['score'].item()
                    true_weight = gF.edges[edge_id].data['weight'].item()
                    predicted_scores.append(predicted_score)
                    true_weights.append(true_weight)
                    f.write(f"{src_name},{dst_name},{predicted_score},{true_weight}\n")


if __name__ == "__main__":
    main()


- Total BCP data: 112
- Total Homopolymer data: 404
Epoch 0, Loss: 3163.3906, Val Loss: 4351.3379
Epoch 20, Loss: 1.5316, Val Loss: 1.3081
Epoch 40, Loss: 1.4146, Val Loss: 1.3524
Epoch 60, Loss: 1.3860, Val Loss: 1.3748
Epoch 80, Loss: 1.3863, Val Loss: 1.3936
Epoch 100, Loss: 1.3860, Val Loss: 1.3954
Epoch 120, Loss: 1.3866, Val Loss: 1.3959
Epoch 140, Loss: 1.3861, Val Loss: 1.3963
Epoch 160, Loss: 1.3852, Val Loss: 1.3965
Epoch 180, Loss: 1.3859, Val Loss: 1.3968
Epoch 200, Loss: 1.3857, Val Loss: 1.3970
Epoch 220, Loss: 1.3859, Val Loss: 1.3969
Epoch 240, Loss: 1.3860, Val Loss: 1.3970
Epoch 260, Loss: 1.3859, Val Loss: 1.3969
Epoch 280, Loss: 1.3866, Val Loss: 1.3968
Epoch 300, Loss: 1.3856, Val Loss: 1.3966
Epoch 320, Loss: 1.3868, Val Loss: 1.3966
Epoch 340, Loss: 1.3869, Val Loss: 1.3966
Epoch 360, Loss: 1.3858, Val Loss: 1.3966
Epoch 380, Loss: 1.3861, Val Loss: 1.3966
Epoch 400, Loss: 1.3862, Val Loss: 1.3966
Epoch 420, Loss: 1.3863, Val Loss: 1.3967
Epoch 440, Loss: 1.3861,

In [179]:
predicted_scores = []
true_weights = []

# Save predicted edges (for inspection) and collect scores and weights
with open('predicted_edges.txt', 'w') as f:
    f.write("Source,Target,Score,Weight\n")
    for edge_id, (src_name, dst_name) in edge_to_str_name_mapping.items():
        predicted_score = gF.edges[edge_id].data['score'].item()
        true_weight = gF.edges[edge_id].data['weight'].item()
        predicted_scores.append(predicted_score)
        true_weights.append(true_weight)
        f.write(f"{src_name},{dst_name},{predicted_score},{true_weight}\n")

# Define the threshold and tolerance
threshold = 800.0
tolerance = 0

amb_lower = threshold - tolerance
amb_upper = threshold + tolerance

fuzzy_correct = 0
for pred, true in zip(predicted_scores, true_weights):
    true_class = 1 if true > threshold else 0
    pred_class = 1 if pred > threshold else 0

    if amb_lower <= pred <= amb_upper:
        fuzzy_correct += 1
    else:
        if pred_class == true_class:
            fuzzy_correct += 1

fuzzy_accuracy = fuzzy_correct / len(true_weights)
print(f"Classification Accuracy : {fuzzy_accuracy:.4f}")


Classification Accuracy : 1.0000


In [ ]:
!pip install dgl
!pip install rdkit

In [12]:
def DataExtNGH():
  dfH = pd.read_excel('database.xlsx','Homopolymers')
  # dfH = pd.read_excel('homopolymers_overall_dp.xlsx')
  # df = df[df['Length'] <= args.size].reset_index(drop=True)
  print('- Total data:', dfH.shape[0])

  # try:
  #     df = df.sample(n=111).reset_index(drop=True)
  #     # dfH = dfH.sample(n=285).reset_index(drop=True)
  #     print('- Sampled data:', df.shape[0])
  # except:
  #     print(f'Sampling error: Set the value of --data lower than {df.shape[0]}.')
  #     quit()

  import numpy as np

  imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

  #*************************************************************************
  # #Normalize Part
  # XE=df ['σbreak (MPa)']
  # XE1=df ['εbreak (%)']
  # # Normalize 'σbreak (MPa)'
  # XE_normalized = (XE - XE.min()) / (XE.max() - XE.min())
  # # Normalize 'εbreak (%)'
  # XE1_normalized = (XE1 - XE1.min()) / (XE1.max() - XE1.min())
  # print(XE1.min(),XE1.max())
  # XE=[XE_normalized]
  # XE1=[XE1_normalized]
  # #*************************************************************************

  # df['Lower_Tg'], df['Upper_Tg']=X[0],X[1]
  Sm=[]
  Cl=[]
  k=-1

  features =[]
  features1=[]
  features2=[]
  import numpy as np

  GH=nx.DiGraph()


  #******************************
  SmilesHo=dfH['Big_Smile'].tolist()
  DP=dfH['Overall DP'].tolist()
  HrdSoft=dfH['Hard/Soft'].tolist()
  Count=0
  print(len(SmilesHo))
  print("*********************************")
  for i ,j,z in zip(SmilesHo[0:len(SmilesHo)], DP[0:len(SmilesHo)], HrdSoft[0:len(SmilesHo)]):
#   for i ,j,z in zip(SmilesHo[0:309], DP[0:309], HrdSoft[0:309]):
    # print(len(i.split(',')))
    # print(i,j)
    # print(Chem.MolFromSmiles(i))
    if len(i.split(','))==1:

      data = [Chem.MolFromSmiles(i)]
      atom_labels = sorted(set([atom.GetAtomicNum() for mol in data for atom in mol.GetAtoms()] + [0]))
      bond_labels = [Chem.rdchem.BondType.ZERO] + list(sorted(set(bond.GetBondType() for mol in data for bond in mol.GetBonds())))
      bond_encoder_m = {l: ii for ii, l in enumerate(bond_labels)}

      Feat = []
      for idx, mol in enumerate(data): # Use enumerate to get the index of the molecule
          combined_feature = torch.cat([feature_size(mol, atom_labels), graph_adjacency(mol, bond_encoder=bond_encoder_m)], 1)

          # Weight the features by the square root of the DP value
          DP_values=j

          dp_weight = torch.sqrt(torch.tensor(float(DP_values), dtype=torch.float))
        #   print(float(DP_values))
          weighted_features = (combined_feature+1) * dp_weight

          Feat.append(weighted_features.float())
      Feat = torch.stack(Feat)
    else:
    #   print(i,j)
      ss=i.split(',')
      jj=j.split('/')
      data = [Chem.MolFromSmiles(ss[0])]
      atom_labels = sorted(set([atom.GetAtomicNum() for mol in data for atom in mol.GetAtoms()] + [0]))
      bond_labels = [Chem.rdchem.BondType.ZERO] + list(sorted(set(bond.GetBondType() for mol in data for bond in mol.GetBonds())))
      bond_encoder_m = {l: ii for ii, l in enumerate(bond_labels)}
      Feat1 = []
      for idx, mol in enumerate(data): # Use enumerate to get the index of the molecule
          combined_feature = torch.cat([feature_size(mol, atom_labels), graph_adjacency(mol, bond_encoder=bond_encoder_m)], 1)

          # Weight the features by the square root of the DP value
          DP_values=jj[0]
          dp_weight = torch.sqrt(torch.tensor(float(DP_values), dtype=torch.float))
          weighted_features = (combined_feature+1) * dp_weight

          Feat1.append(weighted_features.float())

      data = [Chem.MolFromSmiles(ss[1])]
      atom_labels = sorted(set([atom.GetAtomicNum() for mol in data for atom in mol.GetAtoms()] + [0]))
      bond_labels = [Chem.rdchem.BondType.ZERO] + list(sorted(set(bond.GetBondType() for mol in data for bond in mol.GetBonds())))
      bond_encoder_m = {l: ii for ii, l in enumerate(bond_labels)}
      Feat2 = []
      for idx, mol in enumerate(data): # Use enumerate to get the index of the molecule
          combined_feature = torch.cat([feature_size(mol, atom_labels), graph_adjacency(mol, bond_encoder=bond_encoder_m)], 1)

          # Weight the features by the square root of the DP value
          DP_values=jj[1]
          dp_weight = torch.sqrt(torch.tensor(float(DP_values), dtype=torch.float))
          weighted_features = (combined_feature+1) * dp_weight

          Feat2.append(weighted_features.float())
      Feat=torch.stack(Feat1)+torch.stack(Feat2)
      # Feat= torch.cat((torch.stack(Feat1), torch.stack(Feat2)), dim=2)


    padded_features = []
    # print(Feat.shape)
    Feat_M = torch.reshape(Feat, (-1,))
    # print(Feat.shape)

    # print("feat.shape",Feat_M.shape)
    padding_length = 4000 - Feat_M.shape[0]
    Feat_M_padded = torch.cat([Feat_M, torch.zeros(padding_length)])

    padded_features.append(Feat_M_padded)
    ss=0
    padded_features = torch.stack(padded_features)
    Count=Count+1
    padded_features = torch.reshape(padded_features, (-1,))

    # print(padded_features.shape)
    GH.add_node(i+';'+str(j) ,Feature=padded_features,bipartite=int(z),name=Count)




  # Create a mapping from the original string labels to integers
  mapping = {node: i for i, node in enumerate(G.nodes())}
  # Save the reverse mapping to map back to string labels later if needed
  reverse_mapping = {i: node for node, i in mapping.items()}

  # G_combined = nx.compose(G, GH)
  gH=dgl.from_networkx(GH, node_attrs=['Feature','bipartite','name'])
  BH=list(GH.nodes())

  # print(gH.ndata['Feature'].shape)
  return reverse_mapping,gH,GH
reverse_mapping,gH,GH=DataExtNGH()

- Total data: 404
404
*********************************


In [13]:
def combine_graphs(g, gH):
    num_nodes_g = g.number_of_nodes()
    num_nodes_gH = gH.number_of_nodes()

    # Check for missing names in g
    print("Nodes without names in g:", [i for i in range(g.number_of_nodes()) if 'name' not in g.ndata or g.ndata['name'][i] == ''])

    # Check for missing names in gH
    print("Nodes without names in gH:", [i for i in range(gH.number_of_nodes()) if 'name' not in gH.ndata or gH.ndata['name'][i] == ''])

    # Get edges from g
    src_g, dst_g = g.edges()

    # Get edges from gH and shift the node IDs to avoid overlap with g
    src_gH, dst_gH = gH.edges()
    src_gH = src_gH + num_nodes_g  # Shift gH's node indices
    dst_gH = dst_gH + num_nodes_g  # Shift gH's node indices

    # Combine the edges of g and gH
    combined_src = torch.cat([src_g, src_gH])
    combined_dst = torch.cat([dst_g, dst_gH])

    # Create the combined graph
    combined_graph = dgl.graph((combined_src, combined_dst), num_nodes=num_nodes_g + num_nodes_gH)

    # Combine node features (e.g., 'Feature', 'bipartite', etc.)
    if 'Feature' in g.ndata and 'Feature' in gH.ndata:
        combined_graph.ndata['Feature'] = torch.cat([g.ndata['Feature'], gH.ndata['Feature']], dim=0)

    if 'bipartite' in g.ndata and 'bipartite' in gH.ndata:
        combined_graph.ndata['bipartite'] = torch.cat([g.ndata['bipartite'], gH.ndata['bipartite']], dim=0)

    # If names exist, combine node names
    if 'name' in g.ndata and 'name' in gH.ndata:
        combined_graph.ndata['name'] = torch.cat([g.ndata['name'], gH.ndata['name']], dim=0)

    return combined_graph

In [15]:
#This one is link prediction with threshold for link part ---Last version!!!!

import itertools
import torch
import dgl
from torch.nn import Module
from torch.nn.functional import cosine_similarity
import dgl.function as fn

# Function to combine graphs g and gH into a single graph
def combine_graphs(g, gH):
    num_nodes_g = g.number_of_nodes()
    num_nodes_gH = gH.number_of_nodes()

    # Check for missing names in g
    print("Nodes without names in g:", [i for i in range(g.number_of_nodes()) if 'name' not in g.ndata or g.ndata['name'][i] == ''])

    # Check for missing names in gH
    print("Nodes without names in gH:", [i for i in range(gH.number_of_nodes()) if 'name' not in gH.ndata or gH.ndata['name'][i] == ''])

    # Get edges from g
    src_g, dst_g = g.edges()

    # Get edges from gH and shift the node IDs to avoid overlap with g
    src_gH, dst_gH = gH.edges()
    src_gH = src_gH + num_nodes_g  # Shift gH's node indices
    dst_gH = dst_gH + num_nodes_g  # Shift gH's node indices

    # Combine the edges of g and gH
    combined_src = torch.cat([src_g, src_gH])
    combined_dst = torch.cat([dst_g, dst_gH])

    # Create the combined graph
    combined_graph = dgl.graph((combined_src, combined_dst), num_nodes=num_nodes_g + num_nodes_gH)

    # Combine node features (e.g., 'Feature', 'bipartite', etc.)
    if 'Feature' in g.ndata and 'Feature' in gH.ndata:
        combined_graph.ndata['Feature'] = torch.cat([g.ndata['Feature'], gH.ndata['Feature']], dim=0)

    if 'bipartite' in g.ndata and 'bipartite' in gH.ndata:
        combined_graph.ndata['bipartite'] = torch.cat([g.ndata['bipartite'], gH.ndata['bipartite']], dim=0)

    # If names exist, combine node names
    if 'name' in g.ndata and 'name' in gH.ndata:
        combined_graph.ndata['name'] = torch.cat([g.ndata['name'], gH.ndata['name']], dim=0)

    return combined_graph

# Function to generate all possible links within the combined graph
# Function to generate all possible links in the combined graph
import itertools

# Function to generate all possible links within the combined graph
def generate_all_possible_links(combined_g, directed=False):
    num_nodes = combined_g.number_of_nodes()

    if directed:
        # For directed edges, use product to include self-loops
        possible_links = list(itertools.product(range(num_nodes), repeat=2))
    else:
        # For undirected edges, use combinations to exclude self-loops
        possible_links = list(itertools.combinations(range(num_nodes), 2))

    return possible_links



class DotPredictor(Module):
    def forward(self, g, h, threshold=0.5):
        g.ndata['h'] = h
        g.apply_edges(fn.u_dot_v('h', 'h', 'score'))  # Using dot product for prediction score
        scores = g.edata['score'][:, 0]

        # Filter edges based on threshold
        valid_edges = (scores > threshold).nonzero().squeeze()
        src, dst = g.edges(valid_edges)
        return src, dst, scores[valid_edges]


# Option: Using cosine similarity for scoring with threshold
def cosine_predictor(h, possible_links, threshold=0.5):
    predicted_links = []
    for src, dst in possible_links:
        score = cosine_similarity(h[src], h[dst], dim=0).item()  # Cosine similarity score
        if score > threshold:  # Apply threshold here
            predicted_links.append((src, dst, score))
    return predicted_links


# Function to predict links for the combined graph
def predict_new_links(combined_g, possible_links, model, threshold=0.5):
    with torch.no_grad():
        h = model(combined_g, combined_g.ndata['Feature'].float())  # Get embeddings from the model

        # Pass the threshold for filtering low-confidence links
        predicted_links = cosine_predictor(h, possible_links, threshold)
        print(f"Number of predicted links above threshold: {len(predicted_links)}")
        return predicted_links


# Function to map the predicted edges back to node names
def map_predicted_edges(combined_g, G_combined, predicted_links):
    edge_to_str_name_mapping = {}

    # Iterate through the predicted links and map them to their node names
    for src, dst, _ in predicted_links:
        src_name = combined_g.ndata['name'][src].item()
        dst_name = combined_g.ndata['name'][dst].item()

        src_name_in_G = None
        dst_name_in_G = None

        # Map the node names in combined_g back to G_combined
        for node, data in G_combined.nodes(data=True):
            if data.get('name') == src_name:
                src_name_in_G = node
            if data.get('name') == dst_name:
                dst_name_in_G = node
            if src_name_in_G is not None and dst_name_in_G is not None:
                break

        # If both node names are found, store the mapping
        if src_name_in_G is not None and dst_name_in_G is not None:
            edge_to_str_name_mapping[(src, dst)] = (src_name_in_G, dst_name_in_G)

    return edge_to_str_name_mapping


# Main workflow
# Combine the graphs g and gH
combined_g = combine_graphs(g, gH)
print(f"Number of nodes in combined graph: {combined_g.number_of_nodes()}")
# Generate all possible links within the combined graph
possible_links = generate_all_possible_links(combined_g)
print(f"Number of possible links generated: {len(possible_links)}")

# Assuming the GraphSAGE model is trained and available as `model`
# Generate all possible links within the combined graph
possible_links = generate_all_possible_links(combined_g, directed=True)  # Adjust directed as needed

# Assuming the GraphSAGE model is trained and available as `model`
# Predict new links
predicted_links = predict_new_links(combined_g, possible_links, model)

# Mapping predicted edges back to their node names
G_combined = nx.compose(G, GH)
edge_to_str_name_mapping = map_predicted_edges(combined_g, G_combined, predicted_links)

# Print or log to verify results
print(f"Number of predicted links: {len(predicted_links)}")
print(f"Number of mapped edges (including predictions): {len(edge_to_str_name_mapping)}")


# Now create the new graph with both original and predicted edges
# Extract the original edges from gH before combining the graphs
original_src, original_dst = gH.edges()
original_src = original_src.tolist()
original_dst = original_dst.tolist()

# Extract predicted edges, adjusting indices back to gH's node range
predicted_edges = [
    (src - g.number_of_nodes(), dst - g.number_of_nodes())
    for src, dst, _ in predicted_links
    if src >= g.number_of_nodes() and dst >= g.number_of_nodes()
]

# Combine original edges and predicted edges
new_src = original_src + [edge[0] for edge in predicted_edges]
new_dst = original_dst + [edge[1] for edge in predicted_edges]

# Create a new graph with both original and predicted edges (number of nodes in gH)
gH_with_new_edges = dgl.graph((new_src, new_dst), num_nodes=gH.number_of_nodes())

# Transfer the node features from the old graph to the new graph
gH_with_new_edges.ndata.update(gH.ndata)

# (Optional) Print to verify if the new graph has the expected number of edges
print(f"Number of edges in the new graph with predicted links: {gH_with_new_edges.number_of_edges()}")


# Map predicted edges back to their names and save them
with open('edges.txt', 'a') as file:
    for edge_id, (str_name_u, str_name_v) in edge_to_str_name_mapping.items():
        src_id, dst_id = gH_with_new_edges.find_edges(edge_id)
        # if gH_with_new_edges.ndata['bipartite'][src_id] != 0 or gH_with_new_edges.ndata['bipartite'][dst_id] != 0:
            # Only print homopolymer-related edges
        if str_name_u.split(';')[0]!=str_name_v.split(';')[0]:
            file.write(f"{str_name_u}* {str_name_v}\n")
# Remove this condition to save all predicted edges
# with open('edges.txt', 'a') as file:
#     for edge_id, (str_name_u, str_name_v) in edge_to_str_name_mapping.items():
#         file.write(f"{str_name_u}* {str_name_v}\n")

print("Predicted links between homopolymers saved to 'edges.txt'.")




Nodes without names in g: []
Nodes without names in gH: []
Number of nodes in combined graph: 549
Number of possible links generated: 150426
Number of predicted links above threshold: 271039
Number of predicted links: 271039
Number of mapped edges (including predictions): 271039
Number of edges in the new graph with predicted links: 135424
Predicted links between homopolymers saved to 'edges.txt'.
